In [1]:
# Simple CNN model for the google hands Dataset
import numpy
import math
import random as rnd
from keras.datasets import cifar10
import numpy as np
np.random.seed(10)

Using TensorFlow backend.


In [2]:
%matplotlib inline

In [3]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from scipy import signal as sg
from PIL import Image

In [4]:
#圖片變向量
def imgfile2arr(fname):
    return np.asarray(Image.open(fname).convert('L'), dtype=np.float32)
def img2array(img):
    return np.asarray(img, dtype=np.float32)
#向量轉圖片
def arr2img(arr):
    return Image.fromarray(np.clip(arr,0,255).round().astype(np.uint8)).convert('L')

In [5]:
##hands record files
## id	age	gender	skinColor	accessories	nailPolish	aspectOfHand	imageName	irregularities
## 0     1    2         3            4          5            6              7            8       
csvfilepath = 'HandInfo.csv'
handimagepath = 'Hands'
## 控制讀取圖檔上限
readimgmax = 5000
getcsvindex = 1    ## 1:年紀  2:性別

In [ ]:
indic = {}
indicexchange = {}
indiclst = []
## 取得分析資料的所有類別
labelindex = 0
with open(csvfilepath, 'r') as fp:
     lines=fp.readlines()
     for line in lines[1:readimgmax]:
         line = line.replace('"', '')
         s=line.split(',')   
         if getcsvindex == 1:
             label=s[1]
         else:
             if s[2] == 'male':
                 label = 1 
             else:
                 label = 0
         if label not in indiclst:
             indiclst.append(label)
             indic[label] = labelindex
             indicexchange[labelindex] = label
             labelindex += 1
print('indic', indic)  
print('len(indic)', len(indiclst))

indic {'20': 1, '29': 7, '22': 3, '23': 5, '27': 0, '36': 9, '26': 4, '30': 6, '54': 11, '25': 10, '21': 2, '43': 8}
len(indic) 12


In [ ]:
import pickle, os
allimagearr = []


with open(csvfilepath, 'r') as fp:
     lines=fp.readlines()
     for line in lines[1:readimgmax]:
         line = line.replace('"', '')
         s=line.split(',')         
         img = Image.open(handimagepath+os.sep+s[7].replace('"', ''))
         if img.mode != 'RGB':
             img = img.convert('RGB')
         img = img.resize((64, 64), resample=Image.LANCZOS)
         raw_featrure= img2array(img)
         #raw_label=np.array([float(int(s[0])),float(s[1]),float(1 if s[2]=='male' else 0)])
         ##s[1] => s[1]:年紀  s[2]:性別
         if getcsvindex == 1:
             raw_label=[indic[s[1]]]
         else:
             ##raw_label=[indic[1 if s[2]=='male' else 0]]
             raw_label=[indic[s[2]]]
         content=[raw_featrure,raw_label]
         allimagearr.append(content);


with open('hands-1000.pkl', 'wb') as fp:
     pickle.dump(allimagearr, fp, True)



In [ ]:
import pickle, os
image_height = 64
image_width  = 64
num_channels = 3  # RGB
imagearr=[]
with open('hands-1000.pkl', 'rb') as fp:
    imagearr=pickle.load(fp)
    print(len(imagearr))
    print(imagearr[0][0].shape)
    #print(imagearr[0][1].shape)
##show first image
img=Image.fromarray(np.clip(imagearr[0][0],0,255).round().astype(np.uint8)).convert('RGB')
plt.imshow(img)

In [ ]:
from sklearn.cross_validation import train_test_split

In [ ]:
trainDataSet, testDataSet = train_test_split(imagearr, test_size=0.2, random_state=42)
print('訓練組數據筆數為：{0}'.format(len(trainDataSet)))
print('測試組數據筆數為：{0}'.format(len(testDataSet)))

In [ ]:
trainData_img, trainData_label = [x[0]/255 for x in trainDataSet], [x[1] for x in trainDataSet]
testData_img, testData_label = [x[0]/255 for x in testDataSet], [x[1] for x in testDataSet]

In [ ]:
#testData_img[0:2]

In [ ]:
trainData_img_normalize = np.asarray(trainData_img)
testData_img_normalize = np.asarray(testData_img)
##trainData_img_normalize.shape
##(2399, 64, 64, 3)
trainData_label_normalize = np.asarray(trainData_label)
testData_label_normalize = np.asarray(testData_label)
##trainData_label_normalize.shape
##(2399, 1)


In [ ]:
from keras.utils import np_utils
trainData_label_normalize_OneHot = np_utils.to_categorical(trainData_label_normalize)
testData_label_normalize_OneHot = np_utils.to_categorical(testData_label_normalize)

sampleV = '''
trainData_label_normalize_OneHot
array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
'''       

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

In [ ]:
model = Sequential()

In [ ]:
#卷積層1與池化層1

In [ ]:
model.add(Conv2D(filters=16,kernel_size=(3,3),
                 input_shape=(64, 64, 3), 
                 activation='relu', 
                 padding='same'))

In [ ]:
model.add(Dropout(rate=0.3))

In [ ]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
#卷積層2與池化層2

In [ ]:
model.add(Conv2D(filters=32, kernel_size=(3, 3),                   
                 activation='relu', padding='same'))

In [ ]:
model.add(Dropout(0.3))

In [ ]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
#Step3	建立神經網路(平坦層、隱藏層、輸出層)

In [ ]:
model.add(Flatten())
model.add(Dropout(rate=0.3))

In [ ]:
model.add(Dense(1024, activation='relu'))
model.add(Dropout(rate=0.3))

In [ ]:
##輸出層  len(indiclst)

In [ ]:
model.add(Dense(len(indiclst), activation='softmax'))

In [ ]:
print(model.summary())

# 載入之前訓練的模型

In [ ]:
passit = '''
try:
    model.load_weights("SaveModel/CNN-smaller_plk.h5")
    print("載入模型成功!繼續訓練模型")
except :    
    print("載入模型失敗!開始訓練一個新模型")
'''

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [ ]:
train_history=model.fit(trainData_img_normalize, trainData_label_normalize_OneHot,
                        validation_split=0.2,
                        epochs=15, batch_size=50, verbose=2)  

In [ ]:
import matplotlib.pyplot as plt
def show_train_history(train_acc,test_acc):
    plt.plot(train_history.history[train_acc])
    plt.plot(train_history.history[test_acc])
    plt.title('Train History')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
def show_Predicted_Probability(y, prediction, x_img_test, Predicted_Probability, i): 
    print('indicexchange', indicexchange)
    print('label:',indicexchange[y[i][0]],
          'predict:',indicexchange[prediction[i]])
    plt.figure(figsize=(2,2))
    plt.imshow(np.reshape(x_img_test[i],(64, 64, 3)))
    plt.show()
    for j in range(10):
        print(indicexchange[j]+
              ' Probability:%1.9f'%(Predicted_Probability[i][j]))

# 評估模型準確率

In [ ]:
scores = model.evaluate(testData_img_normalize, 
                        testData_label_normalize_OneHot, verbose=2)
scores[1]

In [ ]:
prediction=model.predict_classes(testData_img_normalize)

In [ ]:
prediction[0:20]

In [ ]:
Predicted_Probability=model.predict(testData_img_normalize)

In [ ]:
show_Predicted_Probability(testData_label, prediction,
                           testData_img, Predicted_Probability, 0)

In [ ]:
import pandas as pd
print(indic)
pd.crosstab(testData_label_normalize.reshape(-1), prediction,
            rownames=['label'],colnames=['predict'])

In [ ]:
model.save_weights("SaveModel/CNN-smaller_plk.h5")
print("Saved model to disk")